# DEL 3 - Behandle værdata

In [46]:
# Importer bibliotekene vi trenger
import requests
import json
import pandas as pd
from datetime import datetime

# For automatiske tester trenger vi pytest og ipytest
import ipytest
import pytest
ipytest.autoconfig()

***

## Klasser, @dataclass og pydantic

Vi har i forrige del sett på hvordan vi kan bruke `requests` til å hente data fra Locationforecast APIet til Yr. Nå skal vi se litt på hvordan vi kan implementere klasser for å gjøre det enklere å bruke APIet til å hente ut data vi ønsker. Videre ser vi litt på en spesiell type klasse, `dataclass`.

Etterpå skal vi kikke litt på et bibliotek som heter `pydantic`, som brukes blant annet til å validere json data fra api-er.

Under ser du en enkel implementasjon av en klasse for APIet vi har sett på. Gå gjennom koden og kjør cellen.

In [13]:

class LocationForecastCompactApi:
    """Klasse for å hente værdata fra Yr.no sitt Locationforecast compact API"""
    
    def __init__(self, user_agent='TestApp/1.0 (it-test@sanpro.no)'):
        """Initialiserer klassen med basis-URL og headers"""
        self.base_url = "https://api.met.no/weatherapi/locationforecast/2.0/"
        self.base_url_compact = self.base_url + "compact/"
        self.headers = {
            'User-Agent': user_agent
        }

    def status(self):
        """Henter statusmelding fra API-et"""
        url = self.base_url + "status"
        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error fetching status: {response.status_code}")
        
    def get_forecast(self, latitude, longitude):
        """Henter værdata for gitt bredde- og lengdegrad.
        
        Args:
            latitude (float): Breddegrad
            longitude (float): Lengdegrad

        Returns:
            dict: Værdata for den angitte posisjonen
        """
        url = self.base_url_compact + f"?lat={latitude}&lon={longitude}"

        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error fetching forecast: {response.status_code}")
        
api = LocationForecastCompactApi()
print(api.status())
data = api.get_forecast(59.91, 10.75)
print(json.dumps(data, indent=2))


{'last_update': '2025-10-13T05:27:13Z'}
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [
      10.75,
      59.91,
      3
    ]
  },
  "properties": {
    "meta": {
      "updated_at": "2025-10-13T05:27:12Z",
      "units": {
        "air_pressure_at_sea_level": "hPa",
        "air_temperature": "celsius",
        "cloud_area_fraction": "%",
        "precipitation_amount": "mm",
        "relative_humidity": "%",
        "wind_from_direction": "degrees",
        "wind_speed": "m/s"
      }
    },
    "timeseries": [
      {
        "time": "2025-10-13T05:00:00Z",
        "data": {
          "instant": {
            "details": {
              "air_pressure_at_sea_level": 1026.6,
              "air_temperature": 7.4,
              "cloud_area_fraction": 44.4,
              "relative_humidity": 76.8,
              "wind_from_direction": 121.4,
              "wind_speed": 1.4
            }
          },
          "next_12_hours": {
            "summary": {
  

### Oppgave 6a

- Beskriv kort hva klassen gjør. Ser du noen fordeler med å lage en slik klasse når du skal bruke APIet?
- Hvordan lager du en ny instans / objekt av klassen?
- Hvordan bruker du objektet til å sjekke APIets status?
- Hvordan bruker du objektet til å hente værdata for en gitt lokasjon?

__Skriv svaret ditt her__

***

## Egendefinerte klasser for å strukturere data

Vi kan fint bruke dictionary-objekter for å lagre sammensatte data i variabler. Om en bruker logiske nøkkelnavn, er det ganske enkelt å finne frem til de verdiene man trenger. 

### Eksempel: Kontaktinformasjon
La oss ta et eksempel der vi ønsker å lagre kontaktinformasjon. Vi kan gjøre dette med å lagre verdier i en dictionary:

```python
kontakt = {
    'fornavn': 'Bob',
    'etternavn': 'Sandnes',
    'telefon': 51265854,
    'mobil':  40392284,
    'adresse': 'Langgata 1',
    'postnummer': 4306,
    'poststed': 'Sandnes',
    'epost': 'bob.sandnes@domene.no'
}
```

Når vi da skal finne en verdi som telefonnummer til kontakten, kan vi gjøre
dette med å bruke riktig nøkkelverdi: `kontakt['telefon']`. Dette fungerer godt
på enkle data, og i kortere skript der det er lett å holde oversikten. Når koden
blir større kan det være mer oversiktlig å bruke klasser for å organisere data.
Dette gjør også at en enklere kan validere data, og en får mer hjelp av IDE (som
Visual Studio Code) som gir deg hint til hvilke egenskaper som er tilgjengelig.
Du vil også få beskjed med en gang, mens du skriver, om du for eksempel har
stavet en egenskap feil. Hvis jeg prøver å hente ut telefonnummer med å skrive
`kontakt['tellefon']`, vil dette ikke gi noen feilmelding i IDE, men du får feil
når programmet kjører. 

Med bruk av klasser sørger vi også enkelt for at bruk av navn på egenskaper
holder seg likt gjennom hele koden. Kanskje du først lagret e-post-addresser med
nøkkelverdien `e-post` i variabelen `kontaktA`, men senere lagret du e-post med
nøkkelverdi `epost` i variabel `kontaktB`:

```python

kontaktA['e-post'] = 'bob.sandnes@domene.no'

# ... annen kode ...

kontaktB['epost'] = 'kaare@domene.no'

# ... annen kode ...

# Du lagrer begge kontaktvariablene i en kontaktliste
kontakter = [kontaktA, kontaktB]

# Så skal du skrive ut epost til alle kontaktene i listen:
for kontakt in kontakter:
    print(kontakt['epost'])

# Her vil du få en feilmelding, fordi kontaktA brukte nøkkel 'e-post',
# mens kontaktB brukte nøkkel 'epost'
```

La oss se på hvordan dette kunne vært gjort med klasser:

In [2]:
# Her definerer vi en enkel klasse med de egenskapene vi ønsker for en kontakt
# __init__ metoden kjøres for å initialisere nye objekter av klassen,
# og tar inn parametere som lagres som egenskaper til det nye objektet,
# der self refererer til én spesifikk instans av klassen (ett objekt).
class Kontakt:
    def __init__(
        self,
        fornavn,
        etternavn,
        telefon=None,
        mobil=None,
        adresse=None,
        postnummer=None,
        poststed=None,
        epost=None,
    ):
        self.fornavn = fornavn
        self.etternavn = etternavn
        self.telefon = telefon
        self.mobil = mobil
        self.adresse = adresse
        self.postnummer = postnummer
        self.poststed = poststed
        self.epost = epost

# PS: En liten kommentar om formatering. Over har vi skrevet ett parameter
# per linje i __init__ metoden for å gjøre det mer oversiktlig. Hvis man bare
# har 3-4 parametere i parantesen, kan det fint stå på én linje. Her hadde det
# også vært mulig å skrive hele __init__ metoden på én linje, men den ville
# blitt veldig lang og vanskelig å lese.


# For å lagre kontaktinfo i en variabel, kan vi nå opprette et
# nytt objekt av klassen Kontakt. Vi kan bruke parameternavn
# for å gjøre det tydelig hvilke verdier som settes til hvilke egenskaper.
# Dette gjør koden mer lesbar. Når vi bruker parameternavn, trenger vi
# ikke å følge rekkefølgen på parameterne i __init__ metoden.

kontakt = Kontakt(
    fornavn="Bob",
    etternavn="Sandnes",
    epost="bob.sandnes@domene.no",
    telefon=51265854,
    adresse="Langgata 1",
    postnummer=4306,
    poststed="Sandnes",
    mobil=40392284,
)

# Vi kan også bruke posisjonelle argumenter når vi oppretter et nytt objekt.
# Her bruker vi altså ikke parameternavnene, men følger rekkefølgen på parameterne
# i __init__ metoden. Da blir det mer kompakt, men mindre lesbart, og det kan lettere
# snike seg inn en feil der for eksempel telefon og mobil byttes om.
kontakt2 = Kontakt(
    "Carl",
    "Hansen",
    22334455,
    99887766,
    "Kortveien 2",
    5000,
    "Bergen",
    "carl.hansen@domene.no",
)

# Du så kanskje at noen av parameterne i __init__ metoden har
# standardverdien None. Det betyr at vi kan velge å ikke oppgi
# disse parameterne når vi oppretter et nytt objekt. De vil da
# automatisk få verdien None. Her oppretter vi et nytt objekt
# der vi bare oppgir fornavn og etternavn.
kontakt3 = Kontakt("Dina", "Larsen")

# La oss lagre kontaktene i en liste, og skrive dem ut.
# Legg merke til hvordan vi får tilgang til egenskapene
# til hvert objekt ved å bruke punktum-notasjon.
kontakter = [kontakt, kontakt2, kontakt3]
print("Kontakter:")
for k in kontakter:
    print(f"{k.fornavn} {k.etternavn}, epost: {k.epost}")

Kontakter:
Bob Sandnes, epost: bob.sandnes@domene.no
Carl Hansen, epost: carl.hansen@domene.no
Dina Larsen, epost: None


### Dataclass

I standardbiblioteket til python finnes en spesiell type klasse som heter
`dataclass`. Denne er nyttig å bruke til enkle dataklasser, der
initialiseringskoden er inkludert i klassen. Da slipper vi å skrive
`__init__`-metoden, slik som vi gjorde tidligere. 

For å bruke `dataclass` trenger vi ikke å installere ekstra bibliotek med `pip`,
men vi må importere biblioteket. Kjør cellen under for å importere `dataclass`.

In [3]:
from dataclasses import dataclass

***
### Et lite sidespor: Decorators
For å bruke `dataclass` må vi benytte oss av noe som heter _decorators_ i
python. En _decorator_ er en slags markør som brukes før en klasse- eller
funksjonsdeklarasjon, og er en linje som starter med symbolet `@`. 

En _decorator_ er egentlig en spesiell type funksjon som tar inn en funksjon
eller en klasse, og utvider eller endrer oppførselen til funksjonen/klassen.

La oss lage et kjapt eksempel for å vise hvordan dette fungerer:

In [7]:
def logg_returverdi(func):
    """Decorator som logger returverdien til en funksjon

    PS: Dette er en såkalt docstring. Den brukes til å dokumentere
    hva funksjonen gjør. Du lager docstrings ved å skrive tekst
    mellom triple anførselstegn. Docstrings kan hentes ut av
    dokumentasjonsverktøy og IDE-er for å vise informasjon om
    funksjonen, så når du fører musen over funksjonsnavnet i en IDE,
    vil du kunne se denne teksten (prøv det!).
    """
    def wrapper(*args, **kwargs):
        """Wrapper-funksjon som logger returverdien til den dekorerte funksjonen.
        
        Kommentar: *args og **kwargs brukes for å ta imot et vilkårlig antall
        posisjonelle og navngitte argumenter, slik at wrapper-funksjonen kan
        kalle den opprinnelige funksjonen med de samme argumentene. *args samler
        alle posisjonelle argumenter i en tuple, mens **kwargs samler alle
        navngitte argumenter i en dictionary. Hvis vi for eksempel har en
        funksjon definert som def min_funksjon(a, b, c=3), og vi kaller den som
        min_funksjon(1, 2, c=4), vil args få verdien (1, 2) og kwargs få verdien
        {'c': 4}.
        """
        # Kall den opprinnelige funksjonen med de opprinnelige argumentene
        result = func(*args, **kwargs)
        # Logg returverdien til konsollen
        print(f"Funksjonen returnerte: {result}")
        # Returner returverdien, slik at oppførselen til den 
        # opprinnelige funksjonen bevares
        return result
    return wrapper

# Nå definerer vi en enkel funksjon som legger sammen to tall,
# og bruker dekoratoren vår til å logge returverdien.
@logg_returverdi
def legg_sammen(a, b):
    """En enkel funksjon som legger sammen to tall og returnerer summen."""
    return a + b

# Kall funksjonen for å se hvordan dekoratoren fungerer
sum_resultat = legg_sammen(5, 7)
print("^^Linjen over ble skrevet av dekorator-funksjonen.^^")
print("\nLa oss skrive ut summen igjen fra hovedprogrammet.")
print("Da ser vi funksjonen legg_sammen fungerer som forventet,")
print("selv om vi har utvidet den med en dekorator:\n")
print(f"Summen er: {sum_resultat}")


Funksjonen returnerte: 12
^^Linjen over ble skrevet av dekorator-funksjonen.^^

La oss skrive ut summen igjen fra hovedprogrammet.
Da ser vi funksjonen legg_sammen fungerer som forventet,
selv om vi har utvidet den med en dekorator:

Summen er: 12


Hvis du kjørte cellen over, vil du se at det ble skrevet ut en linje til
konsollen fra dekoratorfunksjonen, i det `legg_sammen` ble kallet, selv om det
ikke er noen kall til `print` i definisjonen til funksjonen `legg_sammen`. Vi
har altså utvidet funksjonaliteten i `legg_sammen` med en egendefinert
dekorator, `@logg_returverdi`.

> Vil du vite mer?
>
> Du kan lese mer om stjerne-uttrykk (`*`) som `*args` og `**kwargs`?
> Les mer om _unpacking_ og operatoren `*` [her](https://www.geeksforgeeks.org/python/starred-expression-in-python/)
> Eller på [W3Schools](https://www.w3schools.com/python/python_args_kwargs.asp)
>
> Hvis du vil se mer på _decorators_ kan du lese om dette [her](https://www.geeksforgeeks.org/python/decorators-in-python/)
> eller på [W3Schools](https://www.w3schools.com/python/python_decorators.asp)



***
### Kontakt-klassen som `dataclass`

Da har vi sporet av nok, og går tilbake til `dataclass`! La oss redefinere
Kontakt-klassen der vi bruker `dataclass`-dekoratøren:

In [ ]:
@dataclass
class Kontakt:
    fornavn: str
    etternavn: str
    telefon: int = None
    mobil: int = None
    adresse: str = None
    postnummer: int = None
    poststed: str = None
    epost: str = None

Dette var mye mer kompakt! Men vent litt - det er noe ekstra greier her med 
`: int ` og `: str`... Dette er noe som heter type hints, men før vi ser på
 det, tar vi en liten repetisjon på datatyper og typing i python.

***
### Litt om datatyper og _Type Hints_

#### Typing og datatyper i python

Som du har lært tidligere, er python et _dynamisk typet_ språk, som vil si at
python bestemmer en variabels datatype under kjøring, slik at programmereren
ikke kan bestemme på forhånd, i koden, hvilken datatype en variabel skal ha.
Videre gjøres typingen implisitt, som også kan kalles _duck typing_: "If it
looks like a duck and quacks like a duck, it's a duck". Med andre ord, verdien 
som lagres i en variabel bestemmer hvilken datatype variabelen får.

Selv om python er veldig fleksibelt når det kommer til variabler, deklarasjoner
og datatyper, er språket samtidig i bunn og grunn _strongly typed_. Det betyr
at du vil få feilmeldinger når du prøver å gjøre operasjoner på blandede datatyper.
For eksempel vil du få feilmelding om du prøver å legge sammen tekst og tall:

```python
# Følgende vil gi TypeError:
to_pluss_to = "2" + 2

# Python har ingen måte å håndtere denne operasjonen på.
# Rent logisk ser vi også at det ikke finnes noen fornuftig standard her,
# da + mellom to streng-verdier er en append-funksjon, mens + mellom to
# tall er en matematisk operasjon.

# Hvis vi tolker begge verdiene som tekst, vil vi få:
to_pluss_to = "2" + str(2)
# som gir resultatet "22"

# Hvis vi tolker begge verdiene som tall, vil vi få:
to_pluss_to = int("2") + 2
# som gir resultatet 4
```

> Kommentar:
>
> Hvordan ser _weak typing_ ut? 
>
> For eksempel kan man i JavaScript, som er svakt typet,
> blande tekst og tall. Da blir en av datatypene automatisk konvertert.
> Så der kan vi skrive `to_pluss_to = "2" + 2`
> uten at dette gir feilmelding, og vil få resultatet `"22"`
> (2 ble konvertert til tekst). For `-` vil tekst bli konvertert til tall,
> så `to_minus_to = "2" - 2` får resultat 0. 
>
> Dette kan være praktisk i noen sammenhenger, men kan også lede til 
> uventede resultater og feil som blir vanskelige å spore.

Selv om python er _strongly typed_ vil du kunne blande noen typer uten å
konvertere dem først. Dette gjelder spesielt i sammenheng med tallrelaterte
datatyper, som `int` og `float`, der også `bool` kan blandes med tall-typer da
`bool` faktisk er en underklasse av `int`. Så når du jobber med tall i python 
vil datatyper endres dynamisk som vist under:

```python
# int + float gir float
resultat = 5 + 2.5      # 7.5 (float)

# int / int, der resultatet ikke er et heltall, gir float
resultat = 10 / 3       # 3.333... (float)

# operasjoner med float, som resulterer i et heltall, 
# vil fortsatt resultere i datatypen float
resultat = 7.5 / 2.5      # 3.0 (float) 
resultat = 2.0 / 2        # 1.0 (float)
resultat = 2.0 + 2        # 4.0 (float)

# bool og int 
resultat = True + 5     # 6
resultat = False * 10   # 0


# Blanding i logiske operasjoner:

# Like tall med ulike datatyper aksepteres
5 == 5.0               # True
True == 1              # True

# Fortsatt ikke mulig å sammenligne tall som tekst med tall
"5" == 5               # False
```

Konverteringen følger hierarkiet `bool → int → float (→ complex)`. Det vil si at
`bool` kan automatisk konverteres til `int` og `int` til `float`, men `float`
vil aldri konverteres automatisk til `int` osv. (`complex` er datatype for
komplekse tall, som er en spesiell type tall i matematikken).

Python følger altså en ganske streng typing, men noen veldefinerte og forutsigbare 
automatiske typekonverteringer gjøres som vist over. Andre språk kan være strengere
når det kommer til datatyper, der for eksempel `5 / 2` vil gi `2` (heltallsdivisjon).
Noen språk vil da kunne tillate `5.0 / 2` som gir flyttallet `2.5`, mens veldig strengt
typede språk vil gi feilmelding ved `5.0 / 2` og bare tillate uttrykk som ikke blander 
flyttall med heltall, som `5.0 / 2.0`.

Så hva gjør du når du vil blande datatyper, eller ha mer kontroll over resultatet fra
operasjoner med ulike datatyper? Kanskje du allerede vet svaret? Du bruker spesielle
operatorer eller eksplisitt konvertering:

```python
# Konverter datatyper før operasjonen gjennomføres:
resultat = int("5") + 3      # 8 (int)
resultat = float("5") + 3    # 8.0 (float)
resultat = int("5") + 3.0    # 8.0 (float)

# Obs - du vil få feilmelding dersom du prøver å gjøre en 
# ugylding konvertering
resultat = int("5.0") + 3    # ValueError ("5.0" kan bare konverteres til float)
resultat = float("5,0") + 3  # ValueError (python gjenkjenner bare '.' som desimaltegn)

# Heltallsdivisjon - noen ganger ønsker vi ikke at resultatet
# av en divisjon skal resultere i et flyttall. Da kan vi bruke
# operatoren // som gjennomfører en heltallsdivisjon:
resultat = 5 // 2            # 2 (int) 
# Eller vi kan konvertere resultatet til int:
resultat = int(5 / 2)        # 2 (int)
# Obs - siden ene tallet er flyttall, vil resultatet få datatypen float,
# selv om heltallsdivisjon brukes:
resultat = 5.0 // 2          # 2.0 (float)
```

> Vil du vite mer?
>
> Les mer om:
>
> - [Duck typing](https://www.geeksforgeeks.org/python/duck-typing-in-python/)
> - [Typing in python - strong, dynamic, implicit](https://medium.com/@pavel.loginov.dev/typing-in-python-strong-dynamic-implicit-c3512785b863)
> - [w3 - Data Types](https://www.w3schools.com/python/python_datatypes.asp)
> - [w3 - Python Casting](https://www.w3schools.com/python/python_casting.asp)


#### _Type Hints_

Dynamisk typing gir stor fleksibilitet og mange fordeler. Samtidig blir det
mindre synlig i koden hvilke datatyper det forventes at ulike variabler skal ha.
Selv om typing i python skjer dynamisk, betyr ikke dette at det er uviktig hvilke 
datatyper som brukes! Og om variabler og parametre mottar data av feil type, vil 
dette raskt lede til feil og krasj av programmet!

For å gjøre koden mer lesbar, der det er enkelt å se hvilke datatyper som
forventes, samt gjøre feilsøking enklere, kan man i python bruke noe som heter
_Type Hints_. Det er en god grunn til at ordet _hint_ er i navnet -
python-interpreteren (programmet som tolker og kjører koden din) ignorerer type
hints fullstendig. Type hints blir altså aldri lest av datamaskinen, og er kun
til støtte for utviklere.

La oss ta et eksempel på en funksjon som beregner arealet av en firkant. Først
har vi funksjonen her, uten type hints:

```python
def beregn_areal(lengde, bredde):
    return lengde * bredde

resultat = beregn_areal(5, 10)
```

Under defineres samme funksjon i en kodecelle, med type hints:

In [13]:
def beregn_areal(lengde: float, bredde: float) -> float:
    """Beregner arealet av et rektangel gitt lengde og bredde.
    
    Kommentar
    ---
    Her bruker vi type hints for å angi at begge parametrene,
    lengde og bredde, skal være av typen float, og at funksjonen
    returnerer en float-verdi.
    """
    return lengde * bredde

resultat: float = beregn_areal(5.0, 10.0)  # Type hints kan også brukes for variabler

print(f"Arealet er: {resultat}")

Arealet er: 50.0


Nå gir vi mer informasjon til utviklere, og til de som skal bruke funksjonen
(som kan være deg selv eller andre). Koden blir mer lesbar, og det er raskere
å vite nøyaktig hvilke datatyper det forventes at funksjonen kan håndtere, 
og hvilken datatype som vil returneres av funksjonen. 

Når du fører musen over funksjonsnavnet, eller etter du skriver funksjonsnavnet
og `(` i IDE, vil du få opp en boks med informasjon om funksjonen. Denne boksen
inkluderer funksjonens header (f.o.m. `def` t.o.m. `:`). For funksjonen vår vil
vi se:

```python
(function) def beregn_areal(
    lengde: float,
    bredde: float
) -> float
```

Merk notasjonene: 
- `:` etter parameter- eller variabelnavn, etterfulgt av navnet på typen.
- `->` mellom funksjonens sluttparantes `)` og `:`, etterfulgt av navnet på
  typen til returverdien.

#### Hva skjer om vi ikke følger type hints?

La oss prøve!

In [21]:
# Her sender vi int i stedet for float
resultat = beregn_areal(5, 10)
print(f"Arealet er (med int): {resultat}")

# Her sender vi en streng i stedet for float
try:
    resultat = beregn_areal("5", "10")
    print(f"Arealet er (med str): {resultat}")
except TypeError as e:
    print(f"Feil: {e}")

# Og et par pussige eksempler
resultat = beregn_areal("5", 10)
print(f"Arealet er (med str og int): {resultat}")

resultat = beregn_areal([5], 10)
print(f"Arealet er (med liste og int): {resultat}")

Arealet er (med int): 50
Feil: can't multiply sequence by non-int of type 'str'
Arealet er (med str og int): 5555555555
Arealet er (med liste og int): [5, 5, 5, 5, 5, 5, 5, 5, 5, 5]


Forklar resultatene etter du kjørte cellen over:
- Hva skjedde med returtypen når begge parametrene var `int`?
- Hva skjenne når begge parametrene var av typen `str`?
    - Hvilken eksakt linje i koden utløste feilen (bruk debug om du er usikker)?
- Hva skjedde når første var `str` og andre var `int` 
    - Hvorfor? 
- Hva skjedde når første var `list` og andre var `int` 
    - Hvorfor? 

_Dobbeltklikk her og skriv inn ditt svar_

Som nevnt bryr ikke datamaskinen seg om type hints! Derfor kjøres koden som om
det ikke var type hints i koden..

Som demonstrert var kanskje ikke type hint i koden over så veldig gode. La oss
se på et litt mer robust eksempel:

In [23]:
def beregn_areal(lengde: float | int, bredde: float | int) -> float | int:
    """Beregner arealet av et rektangel gitt lengde og bredde.
    
    Kommentar
    ---
    Vi kan bruke union types (float | int) for å angi at typen kan være 
    en av de angitte typene, adskilt med en vertikal strek (|).
    """
    # For å gjøre funksjonen mer robust, kan vi sjekke
    # at parametrene har gyldige typer før vi utfører beregningen.
    if not isinstance(lengde, (float, int)):
        raise TypeError(f"Lengde må være av typen float eller int, ikke {type(lengde)}")
    if not isinstance(bredde, (float, int)):
        raise TypeError(f"Bredde må være av typen float eller int, ikke {type(bredde)}")
    
    return lengde * bredde

# Her er det egentlig unødvendig å bruke type hints
# for resultat-variabelen, fordi IDE automatisk skjønner
# at typen skal settes til float | int basert på returtypen
# til funksjonen. Før musen over resultat-variabelen for å se
# at VSCode skjønner hvilken type variabelen forventes å ha.
resultat = beregn_areal(5, 10)

print(f"Arealet er: {resultat}")

# La oss teste igjen med ugyldige typer
try:
    resultat = beregn_areal("5", "10")
except TypeError as e:
    print(f"Feil: {e}")

try:
    resultat = beregn_areal(5, "10")
except TypeError as e:
    print(f"Feil: {e}")

try:
    resultat = beregn_areal([5], 10)
except TypeError as e:
    print(f"Feil: {e}")

Arealet er: 50
Feil: Lengde må være av typen float eller int, ikke <class 'str'>
Feil: Bredde må være av typen float eller int, ikke <class 'str'>
Feil: Lengde må være av typen float eller int, ikke <class 'list'>


Siden ikke python-interpreter leser type hints, kan det være aktuelt at du
skriver kode som sjekker at parametrene som sendes inn i funksjonen er gyldige.
Dette har vi gjort her ved å sjekke datatypen til parametrene, og reise en
egendefinert feilmedling dersom funksjonen mottar en uventet datatype. Det er
ikke helt nødvendig å gjøre dette, og det vil være en vurdering i det konkrete
tilfellet om det er lurt å implementere slik parameterkontroll. 

Robust kode med presis feilhåndtering er imidlertid god kodepraksis, og med
større og mer komplekse programmer kan det fort bli utfordrende å spore feil
dersom man ikke implementerer sjekker og reiser beskrivende feil med forklarende
feilmeldinger. Se eksempel på et enkelt program som bruker `beregn_areal`:

In [43]:
def beregn_areal(lengde: float | int, bredde: float | int) -> float | int:
    """Beregner arealet av et rektangel gitt lengde og bredde.
    
    Kommentar
    ---
    Her har vi type-hints, men har droppet sjekkene for
    parameter-typene.
    """
    return lengde * bredde

def beregn_areal_robust(lengde: float | int, bredde: float | int) -> float | int:
    """Beregner arealet av et rektangel gitt lengde og bredde.
    
    Kommentar
    ---
    Her er den mer robuste versjonen vist tidligere, 
    med sjekk av parameter-typene.
    """
    if not isinstance(lengde, (float, int)):
        raise TypeError(f"Lengde må være av typen float eller int, ikke {type(lengde)}")
    if not isinstance(bredde, (float, int)):
        raise TypeError(f"Bredde må være av typen float eller int, ikke {type(bredde)}")
    
    return lengde * bredde

def beregn_areal_fra_input(beregn_areal=beregn_areal):
    """Beregner arealet av et rektangel basert på brukerinput."""
    print("--- Beregn areal for rektangel ---\n")
    bredde = int(input("Skriv inn bredde:"))
    lengde = input("Skriv inn lengde:")

    areal = beregn_areal(lengde, bredde)
    print(f"\nArealet av rektangel med lengde {lengde} og bredde {bredde} er:\n{areal}")

Over har vi definert `beregn_areal`, der vi ikke har implementert sjekk av
parameter-typene. `beregn_areal_robust` har de samme typesjekkene som over. Funksjonen `beregn_areal_fra_input` kjører et kort skript som
skal ta inn brukerdata (lengde og bredde) med `input()`, og skrive ut arealet
av et rektangel med slik lengde og bredde. I funksjonen er det sneket seg inn en liten
feil, som gjør at vi får et litt interessant resultat...

Kjør cellen over, og så den under som simulerer brukerinput. Det er ikke så viktig å se på eller forstå koden i cellen under, fokuser på output fra cellen etter du kjører den.

In [68]:
%%ipytest -s

def mocked_input(input_iter, prompt):
    svar = next(input_iter)
    print(f"{prompt.strip()} {svar}")
    return svar

def test_beregn_areal_fra_input_ok(monkeypatch):
    print("\n#### Kjører test med beregn_areal, inndata 1 og 5 ####\n")
    input_iter = iter(["1", "5"])
    monkeypatch.setattr('builtins.input', lambda prompt: mocked_input(input_iter, prompt))
    beregn_areal_fra_input()
    print("\n-----\nKommentar: Dette ser jo riktig ut?")

def test_beregn_areal_fra_input(monkeypatch):
    print("\n#### Kjører test med beregn_areal, inndata 10 og 5 ####\n")
    input_iter = iter(["10", "5"])
    monkeypatch.setattr('builtins.input', lambda prompt: mocked_input(input_iter, prompt))
    beregn_areal_fra_input()
    print("\n-----\nKommentar: Her er det noe som ikke stemmer - men vi får ingen feilmelding!")

def test_beregn_areal_fra_input_robust(monkeypatch):
    print("\n#### Kjører test med beregn_areal_robust, inndata 10 og 5 ####\n")
    input_iter = iter(["10", "5"])
    monkeypatch.setattr('builtins.input', lambda prompt: mocked_input(input_iter, prompt))
    with pytest.raises(TypeError) as exc_info:
        beregn_areal_fra_input(beregn_areal=beregn_areal_robust)
    print("\n---------------\nProgrammet ble avsluttet.\nFølgende TypeError ble reist:")
    print(exc_info.value)
    print("Stack trace:")
    for trace in exc_info.traceback:
        print(trace)
    print("\n-----\nKommentar: Her får vi en feilmelding som forteller oss hva som gikk galt!")



#### Kjører test med beregn_areal, inndata 1 og 5 ####

--- Beregn areal for rektangel ---

Skriv inn bredde: 1
Skriv inn lengde: 5

Arealet av rektangel med lengde 5 og bredde 1 er:
5

-----
Kommentar: Dette ser jo riktig ut?
.
#### Kjører test med beregn_areal, inndata 10 og 5 ####

--- Beregn areal for rektangel ---

Skriv inn bredde: 10
Skriv inn lengde: 5

Arealet av rektangel med lengde 5 og bredde 10 er:
5555555555

-----
Kommentar: Her er det noe som ikke stemmer - men vi får ingen feilmelding!
.
#### Kjører test med beregn_areal_robust, inndata 10 og 5 ####

--- Beregn areal for rektangel ---

Skriv inn bredde: 10
Skriv inn lengde: 5

---------------
Programmet ble avsluttet.
Følgende TypeError ble reist:
Lengde må være av typen float eller int, ikke <class 'str'>
Stack trace:
  File 'C:\Users\bjarte\AppData\Local\Temp\ipykernel_38388\1747889447.py':25 in test_beregn_areal_fra_input_robust
  beregn_areal_fra_input(beregn_areal=beregn_areal_robust)

  File 'C:\Users\bjarte\App

Over skal du se output fra tre tester. Først med bruker-input 1 og 5, så med brukerinput 10 og 5, og til slutt brukerinput 10 og 5 med den robuste implementeringen. 

> Merk:
>
> Det kan være du må trykke på lenken _scrollable element_ for å se hele utskriften over

__Tenk gjennom og svar i tekstboksen under:__
- Hvorfor ser svaret riktig ut i den første testen?
- Hvorfor blir svaret feil i den andre testen
    - Hva kan være utfordringene med å spore denne feilen?
- Hvorfor får vi feilmelding på den siste testen?
    - Hvordan gjør feilmeldingen det enklere å oppdage og spore feilen?

_Dobbeltklikk her og skriv inn ditt svar_

In [60]:
# Kjør denne cellen om du vil teste hvordan beregn_areal_fra_input UTEN robust
# typesjekking fungerer, der du kan skrive inn verdier for lengde og bredde
# selv.
beregn_areal_fra_input()

--- Beregn areal for rektangel ---


Arealet av rektangel med lengde 4 og bredde 1 er:
4


In [59]:
# Kjør denne cellen om du vil teste hvordan beregn_areal_fra_input MED robust
# typesjekking fungerer, der du kan skrive inn verdier for lengde og bredde
# selv.
beregn_areal_fra_input(beregn_areal=beregn_areal_robust)

--- Beregn areal for rektangel ---



TypeError: Lengde må være av typen float eller int, ikke <class 'str'>

***

### Tilbake til `dataclass`

Da har vi tatt en litt lengre omvei innom datatyper og type hints, og det er på tide at vi kikker videre på `dataclass`. Under er klassen som vi så vidt så på tidligere:

In [69]:
@dataclass
class Kontakt:
    fornavn: str
    etternavn: str
    telefon: int = None
    mobil: int = None
    adresse: str = None
    postnummer: int = None
    poststed: str = None
    epost: str = None

# Denne klassen kan vi bruke på samme måte som Kontakt-klassen vi definerte tidligere:

kontakt = Kontakt(
    fornavn="Bob",
    etternavn="Sandnes",
    epost="bob.sandnes@domene.no",
    telefon=51265854,
    adresse="Langgata 1",
    postnummer=4306,
    poststed="Sandnes",
    mobil=40392284,
)
kontakt2 = Kontakt(
    "Carl",
    "Hansen",
    22334455,
    99887766,
    "Kortveien 2",
    5000,
    "Bergen",
    "carl.hansen@domene.no",
)
kontakt3 = Kontakt("Dina", "Larsen")

kontakter = [kontakt, kontakt2, kontakt3]
print("Kontakter:")
for k in kontakter:
    print(f"{k.fornavn} {k.etternavn}, epost: {k.epost}")

Kontakter:
Bob Sandnes, epost: bob.sandnes@domene.no
Carl Hansen, epost: carl.hansen@domene.no
Dina Larsen, epost: None


### Nøstede dataklasser

Vi kan definere flere dataklasser som videre klassifiserer og samler data, for eksempel ved å lage en egen klasse for Adresse.

In [ ]:


@dataclass
class Adresse:
    gate: str
    husnummer: str
    postnummer: int
    poststed: str
    land: str | None = None

@dataclass
class Kontakt:
    fornavn: str
    etternavn: str
    telefon: int | None = None
    mobil: int | None = None
    adresse: Adresse | None = None
    epost: str | None = None

kontakt = Kontakt(
    fornavn="Eva",
    etternavn="Nilsen",
    telefon=33445566,
    mobil=99887766,
    adresse=Adresse(
        gate="Hovedgata",
        husnummer="10B",
        postnummer=1234,
        poststed="Oslo",
        land="Norge"
    ),
    epost="eva.nilsen@example.com"
)

***

## Validering med `pydantic`

In [15]:

class CompactForecastResponse:
    """Klasse for å representere et svar fra Yr.no sitt Locationforecast compact API"""
    
    def __init__(self, data):
        """Initialiserer en CompactForecastResponse-instans fra et API-svar"""
        self.type = data['type']
        self.geometry = data['geometry']
        self.metadata = data['properties']['meta']
        self.timeseries = data['properties']['timeseries']

In [16]:

class LocationForecastCompactApi:
    """Klasse for å hente værdata fra Yr.no sitt Locationforecast compact API"""
    
    def __init__(self, user_agent='TestApp/1.0 (it-test@sanpro.no)'):
        """Initialiserer en LocationForecastApi-instans med bredde- og lengdegrad"""
        self.base_url = "https://api.met.no/weatherapi/locationforecast/2.0/"
        self.base_url_compact = self.base_url + "compact/"
        self.headers = {
            'User-Agent': user_agent
        }

    def status(self):
        """Henter statusmelding fra API-et"""
        url = self.base_url + "status"
        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"Error fetching status: {response.status_code}")
        
    def get_forecast(self, latitude, longitude):
        """Henter værdata for gitt bredde- og lengdegrad.
        
        Args:
            latitude (float): Breddegrad
            longitude (float): Lengdegrad

        Returns:
            dict: Værdata for den angitte posisjonen
        """
        url = self.base_url_compact + f"?lat={latitude}&lon={longitude}"

        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            data = response.json()
            return CompactForecastResponse(data)
        else:
            raise Exception(f"Error fetching forecast: {response.status_code}")

api = LocationForecastCompactApi()
data = api.get_forecast(59.91, 10.75)
print(json.dumps(data.timeseries, indent=2, ensure_ascii=False))

NameError: name 'requests' is not defined

***

## Oppgave 6: Lag en klasse for værdata

Nå som vi forstår datastrukturen, kan vi enklere hente ut data vi ønsker.

### Oppgave 6a - Lag en klasse for værdata

Lag en klasse, `Prognosis`, som

***

## Oppgave 6: Lag en klasse for værdata

Nå som vi forstår datastrukturen, kan vi lage en klasse for å jobbe med værdataene på en enkel måte.

### Oppgave 6a - Lag en enkel Prognosis-klasse

Lag en klasse `Prognosis` som:
- Tar inn et element fra timeseries-listen i konstruktøren
- Lagrer `time` som en instansvariabel av typen datetime
- Lagrer følgende værdata som instansvariabler med navnene:
    - Lufttrykk: `air_pressure`
    - Temperatur: `air_temperature`
    - Skydekke: `cloud_fraction`
    - Fuktighet: `humidity`
    - Vindretning: `wind_direction`
    - Vindhastighet: `wind_speed`

**Eksempel på bruk:**
```python
prognoser = get_timeseries()
første_prognose = Prognosis(prognoser[0])
print(første_prognose.time)  # Skriver ut tidspunktet
```

In [ ]:
class Prognosis:
    """Klasse for å representere en værprognose fra Yr.no API"""
    
    def __init__(self, timeseries_entry):
        """Initialiserer en Prognosis-instans fra et timeseries-objekt"""
        pass  # Erstatt denne linjen med din kode
        self.time = datetime.fromisoformat(timeseries_entry['time'])
        self.air_pressure = timeseries_entry['data']['instant']['details'].get('air_pressure_at_sea_level')
        self.air_temperature = timeseries_entry['data']['instant']['details'].get('air_temperature')
        self.cloud_fraction = timeseries_entry['data']['instant']['details'].get('cloud_fraction')
        self.humidity = timeseries_entry['data']['instant']['details'].get('relative_humidity')
        self.wind_direction = timeseries_entry['data']['instant']['details'].get('wind_from_direction')
        self.wind_speed = timeseries_entry['data']['instant']['details'].get('wind_speed')


_Kjør cellen under for å teste klassen din_

In [ ]:
%%ipytest

def test_prognosis_init():
    prognoser = get_timeseries()
    prognose = Prognosis(prognoser[0])
    
    assert isinstance(prognose.time, datetime), "prognose.time skal være av typen datetime."
    assert isinstance(prognose.air_pressure, (float, type(None))), "prognose.air_pressure skal være float eller None."
    assert isinstance(prognose.air_temperature, (float, type(None))), "prognose.air_temperature skal være float eller None."
    assert isinstance(prognose.cloud_fraction, (float, type(None))), "prognose.cloud_fraction skal være float eller None."
    assert isinstance(prognose.humidity, (float, type(None))), "prognose.humidity skal være float eller None."
    assert isinstance(prognose.wind_direction, (float, type(None))), "prognose.wind_direction skal være float eller None."
    assert isinstance(prognose.wind_speed, (float, type(None))), "prognose.wind_speed skal være float eller None."


***

### Oppgave 6b - bruk Prognosis

Bruk klassen du lagde, `Prognosis`, til å holde værdata hentet fra API. Bruk `get_timeseries()` for å hente en tidsserie, og last den første prognosen inn i et nytt `Prognosis` objekt med variabelnavn `first_prognosis`

In [ ]:
time_series = get_timeseries()
first_element = time_series[0]
first_prognosis = Prognosis(first_element)

_Kjør cellen under for å sjekke svaret ditt_

In [ ]:
%%ipytest

def test_first_prognosis():
    assert isinstance(first_prognosis, Prognosis), "first_prognosis skal være en instans av Prognosis."
    expected_timestamp = datetime.fromisoformat(get_timeseries()[0]['time'])
    assert first_prognosis.time == expected_timestamp, "first_prognosis.time har ikke riktig verdi."

Bruk `first_prognosis` for å skrive ut (print) temperatur og vindhastighet

In [ ]:
def print_weather_info():
    """Skriv ut temperatur og vindhastighet fra en Prognosis-instans"""
    pass # Erstatt denne linjen med din kode
    print(f"Temperatur: {first_prognosis.air_temperature} °C")
    print(f"Vindhastighet: {first_prognosis.wind_speed} m/s")

print_weather_info()

In [ ]:
%%ipytest

def test_print_weather_info(capsys):
    print_weather_info()
    captured = capsys.readouterr()
    prognosis = get_timeseries()[0]['data']['instant']['details']
    assert str(prognosis['air_temperature']) in captured.out, "Temperaturdata mangler i utskriften."
    assert str(prognosis['wind_speed']) in captured.out, "Vindhastighetsdata mangler i utskriften."

---

### Oppgave 6c - Lag en enkel Timeseries-klasse



In [ ]:
class Timeseries:
    """Klasse for å representere en tidsserie fra Yr.no API"""
    
    def __init__(self, timeseries):
        """Initialiserer en Timeseries-instans fra et timeseries-objekt"""
        self.entries = [Prognosis(entry) for entry in timeseries['properties']['timeseries']]
    
    def get_temperature(self):
        """Henter lufttemperatur i celsius"""
        pass  # Erstatt denne linjen med din kode
    
    def get_wind_speed(self):
        """Henter vindhastighet i m/s"""
        pass  # Erstatt denne linjen med din kode


_Kjør cellen under for å teste metodene dine_

In [ ]:
%%ipytest

def test_prognosis_methods():
    prognoser = get_timeseries()
    prognose = Prognosis(prognoser[0])
    
    # Test at metodene returnerer tall
    temp = prognose.get_temperature()
    wind = prognose.get_wind_speed()
    
    assert isinstance(temp, (int, float)), "get_temperature() skal returnere et tall"
    assert isinstance(wind, (int, float)), "get_wind_speed() skal returnere et tall"
    
    # Test at verdiene er rimelige
    assert -50 < temp < 50, "Temperatur skal være mellom -50 og 50°C"
    assert 0 <= wind < 100, "Vindhastighet skal være mellom 0 og 100 m/s"


---

La oss teste klassen vår med ekte data:

In [ ]:
# Hent prognoser og lag Prognosis-objekter
prognoser = get_timeseries()

# Vis de 5 første prognosene
print("De 5 første værmeldingene:\n")
for i in range(5):
    p = Prognosis(prognoser[i])
    print(f"{p.time}: {p.get_temperature()}°C, vind {p.get_wind_speed()} m/s")

---

### Oppgave 6c - Finn høyeste og laveste temperatur

Skriv kode som:
1. Går gjennom alle prognoser i timeseries
2. Finner den høyeste temperaturen
3. Finner den laveste temperaturen
4. Skriver ut resultatene

**Tips:** Bruk en løkke og to variabler for å holde styr på min og max temperatur.

In [ ]:
# Skriv din kode her
prognoser = get_timeseries()

# Initialiser variabler for å holde styr på min/max
# ... din kode

# Gå gjennom alle prognoser
# ... din kode

# Skriv ut resultatet
# ... din kode


***

## Oppgave 7: Visualiser værdata med pandas

Nå skal vi bruke pandas til å analysere og visualisere værdataene på en mer strukturert måte.

### Oppgave 7a - Lag en DataFrame med værdata

Lag en pandas DataFrame som inneholder:
- Tidspunkt (time)
- Temperatur (temperature)
- Vindhastighet (wind_speed)

**Tips:** Bruk en liste med dictionaries og `pd.DataFrame()`

In [ ]:
# Lag en liste med værdata
prognoser = get_timeseries()
weather_data = []

for entry in prognoser:
    p = Prognosis(entry)
    weather_data.append({
        'time': p.time,
        'temperature': p.get_temperature(),
        'wind_speed': p.get_wind_speed()
    })

# Lag DataFrame
df = pd.DataFrame(weather_data)

# Konverter time til datetime
df['time'] = pd.to_datetime(df['time'])

# Vis de første radene
print(df.head())
print(f"\nAntall prognoser: {len(df)}")

---

### Oppgave 7b - Grunnleggende statistikk

Bruk pandas' innebygde metoder til å finne:
- Gjennomsnittstemperatur
- Gjennomsnittlig vindhastighet
- Minimum og maksimum temperatur

In [ ]:
# Skriv din kode her for å finne statistikk

# Tips: Bruk df.describe() eller df['kolonnenavn'].mean(), df['kolonnenavn'].min(), etc.


**Spørsmål:**
- Hva er gjennomsnittstemperaturen for de neste dagene?
- Hva er den høyeste og laveste temperaturen som er meldt?

_Skriv svaret ditt her_

***

## Gratulerer! 🎉

Du har nå:
- Lært hvordan man bruker et REST API
- Jobbet med JSON-data
- Laget en klasse for å strukturere data
- Brukt pandas til dataanalyse

### Bonusoppgaver (valgfritt)

Hvis du vil utfordre deg selv videre, prøv å:
1. Legge til flere metoder i Prognosis-klassen (f.eks. `get_humidity()`, `get_precipitation()`)
2. Lage et plott av temperatur over tid ved hjelp av matplotlib eller plotly
3. Sammenligne været for flere forskjellige lokasjoner
4. Hente data fra Yr sitt `complete` API i stedet for `compact` og se hva som er forskjellen

## Oppgave 6: Hent værprognose for flere timer

API-et gir oss prognoser for mange timer fremover. La oss lage en funksjon som henter prognoser for de neste timene.

In [ ]:
def get_hourly_forecast(lat, lon, hours=12):
    """
    Henter timesprognose for de neste X timene.
    
    Args:
        lat (float): Breddegrad
        lon (float): Lengdegrad
        hours (int): Antall timer å hente prognose for
    
    Returns:
        list: Liste med værprognoser
    """
    
    headers = {'User-Agent': 'VærApp-Student/1.0 (student@skole.no)'}
    url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={lat}&lon={lon}"
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        
        if response.status_code != 200:
            print(f"API-feil: Status {response.status_code}")
            return []
        
        data = response.json()
        timeseries = data['properties']['timeseries']
        
        forecast = []
        
        # Gå gjennom de første 'hours' prognosene
        for i, entry in enumerate(timeseries[:hours]):
            try:
                instant_data = entry['data']['instant']['details']
                
                forecast_entry = {
                    'tidspunkt': entry['time'],
                    'temperatur': instant_data['air_temperature'],
                    'luftfuktighet': instant_data['relative_humidity'],
                    'vindhastighet': instant_data['wind_speed']
                }
                
                forecast.append(forecast_entry)
                
            except KeyError:
                # Noen tidspunkter kan mangle data
                continue
        
        return forecast
        
    except Exception as e:
        print(f"Feil ved henting av prognose: {e}")
        return []

# Test funksjonen
oslo_forecast = get_hourly_forecast(59.9139, 10.7522, 8)

if oslo_forecast:
    print("📊 8-timers prognose for Oslo:")
    
    # Konverter til DataFrame for pen visning
    df = pd.DataFrame(oslo_forecast)
    
    # Forenkle tidspunkt-visning
    df['tid'] = pd.to_datetime(df['tidspunkt']).dt.strftime('%H:%M')
    
    # Vis de første prognosene
    print(df[['tid', 'temperatur', 'luftfuktighet', 'vindhastighet']].head(8).to_string(index=False))
else:
    print("❌ Kunne ikke hente værprognose")

**Oppgave 6a:** Finn temperatur-ekstremene i prognosen

In [ ]:
# Analyser temperaturen i prognosen
if oslo_forecast:
    temperatures = [entry['temperatur'] for entry in oslo_forecast]
    
    min_temp = min(temperatures)
    max_temp = max(temperatures)
    avg_temp = sum(temperatures) / len(temperatures)
    
    print(f"🌡️ Temperaturanalyse for de neste {len(oslo_forecast)} timene:")
    print(f"Laveste: {min_temp}°C")
    print(f"Høyeste: {max_temp}°C")
    print(f"Gjennomsnitt: {avg_temp:.1f}°C")
    print(f"Temperaturspenn: {max_temp - min_temp}°C")

## Oppgave 7: Sammenlign været i flere byer

La oss bruke funksjonene våre til å sammenligne været i flere norske byer.

In [ ]:
# Definer koordinater for norske byer
norske_byer = {
    'Oslo': (59.9139, 10.7522),
    'Bergen': (60.3913, 5.3221),
    'Trondheim': (63.4305, 10.3951),
    'Stavanger': (58.9700, 5.7331),
    'Tromsø': (69.6492, 18.9553)
}

# Hent værdata for alle byene
by_weather = []

print("🌍 Henter værdata for norske byer...")

for by_navn, (lat, lon) in norske_byer.items():
    print(f"  Henter data for {by_navn}...")
    
    weather = get_current_weather(lat, lon)
    
    if weather:
        weather['by'] = by_navn
        by_weather.append(weather)
    else:
        print(f"    ⚠️ Kunne ikke hente data for {by_navn}")

# Vis resultatene
if by_weather:
    print("\n🌤️ Værsammenligning:")
    
    # Konverter til DataFrame
    df = pd.DataFrame(by_weather)
    
    # Vis relevante kolonner
    weather_table = df[['by', 'temperatur_celsius', 'luftfuktighet_prosent', 'vindhastighet_ms']].round(1)
    weather_table.columns = ['By', 'Temperatur (°C)', 'Luftfuktighet (%)', 'Vind (m/s)']
    
    print(weather_table.to_string(index=False))
    
    # Finn ekstremene
    varmeste_idx = df['temperatur_celsius'].idxmax()
    kaldeste_idx = df['temperatur_celsius'].idxmin()
    
    print(f"\n🔥 Varmeste by: {df.loc[varmeste_idx, 'by']} ({df.loc[varmeste_idx, 'temperatur_celsius']}°C)")
    print(f"🧊 Kaldeste by: {df.loc[kaldeste_idx, 'by']} ({df.loc[kaldeste_idx, 'temperatur_celsius']}°C)")
else:
    print("❌ Kunne ikke hente værdata for noen byer")

**Oppgave 7a:** Finn byen med høyest vindhastighet

In [ ]:
# Din kode her:
# Finn byen med høyest vindhastighet
if by_weather:
    # vindigste_idx = df['vindhastighet_ms'].idxmax()
    # print(f"💨 Vindigste by: {df.loc[vindigste_idx, 'by']} ({df.loc[vindigste_idx, 'vindhastighet_ms']} m/s)")
    pass

## Oppgave 8: Avansert - Daglig værsammendrag

La oss lage en funksjon som gir oss et sammendrag av været for de neste dagene.

In [ ]:
def get_daily_summary(lat, lon, days=3):
    """
    Henter daglig værsammendrag for de neste dagene.
    
    Args:
        lat (float): Breddegrad
        lon (float): Lengdegrad
        days (int): Antall dager
    
    Returns:
        DataFrame: Daglig værsammendrag
    """
    
    # Hent prognose for mange timer (24 timer * antall dager)
    forecast = get_hourly_forecast(lat, lon, hours=days * 24)
    
    if not forecast:
        return None
    
    # Konverter til DataFrame
    df = pd.DataFrame(forecast)
    
    # Legg til dato-kolonne
    df['datetime'] = pd.to_datetime(df['tidspunkt'])
    df['dato'] = df['datetime'].dt.date
    
    # Gruppér etter dag og beregn statistikk
    daily_summary = df.groupby('dato').agg({
        'temperatur': ['min', 'max', 'mean'],
        'luftfuktighet': 'mean',
        'vindhastighet': ['mean', 'max']
    }).round(1)

    # Forenkle kolonnenavn
    daily_summary.columns = [
        'Min_temp', 'Max_temp', 'Snitt_temp',
        'Snitt_luftfuktighet', 'Snitt_vind', 'Max_vind'
    ]

    return daily_summary

# Test funksjonen for Oslo
print("📅 3-dagers værsammendrag for Oslo:")
oslo_summary = get_daily_summary(59.9139, 10.7522, 3)

if oslo_summary is not None:
    print(oslo_summary)

    print(f"📊 Analyse:")
    print(f"Varmeste dag: {oslo_summary['Max_temp'].max()}°C")
    print(f"Kaldeste dag: {oslo_summary['Min_temp'].min()}°C")
    print(f"Største temperaturspenn på en dag: {(oslo_summary['Max_temp'] - oslo_summary['Min_temp']).max()}°C")
else:
    print("❌ Kunne ikke hente daglig sammendrag")


## Bonusoppgaver

Hvis du har kommet så langt, kan du prøve disse utfordringsoppgavene!

### Bonusoppgave 1: Lag en værvarsel-klasse

Organiser funksjonaliteten i en Python-klasse for bedre struktur.

In [ ]:
class WeatherAPI:
    """Klasse for å hente værdata fra Yr.no API"""
    
    def __init__(self, user_agent="VærApp-Student/1.0 (student@skole.no)"):
        self.base_url = "https://api.met.no/weatherapi/locationforecast/2.0/compact"
        self.headers = {'User-Agent': user_agent}
    
    def _make_request(self, lat, lon):
        """Hjelpemetode for å gjøre API-forespørsel"""
        url = f"{self.base_url}?lat={lat}&lon={lon}"
        
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()  # Kaster exception ved HTTP-feil
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"API-feil: {e}")
            return None
    
    def get_current_weather(self, lat, lon):
        """Hent nåværende vær"""
        # Implementer denne metoden
        # Din kode her:
        pass
    
    def get_forecast(self, lat, lon, hours=12):
        """Hent timesprognose"""
        # Implementer denne metoden
        # Din kode her:
        pass

# Test klassen din
# weather_api = WeatherAPI()
# weather = weather_api.get_current_weather(59.9139, 10.7522)
# print(weather)

### Bonusoppgave 2: Robust feilhåndtering

Implementer en funksjon som prøver på nytt hvis API-et ikke svarer.

In [ ]:
import time

def robust_weather_request(lat, lon, max_retries=3, delay=1):
    """
    Robust API-forespørsel med retry-logikk.
    
    Args:
        lat, lon: Koordinater
        max_retries: Maksimalt antall forsøk
        delay: Sekunder å vente mellom forsøk
    """
    
    for attempt in range(max_retries):
        try:
            print(f"Forsøk {attempt + 1}/{max_retries}...")
            
            # Din kode her:
            # Implementer logikk som:
            # 1. Prøver å hente data
            # 2. Returnerer data hvis vellykket
            # 3. Venter 'delay' sekunder og prøver igjen ved feil
            # 4. Gir opp etter max_retries forsøk
            
            pass
            
        except Exception as e:
            print(f"Forsøk {attempt + 1} feilet: {e}")
            
            if attempt < max_retries - 1:
                print(f"Venter {delay} sekunder før nytt forsøk...")
                time.sleep(delay)
            else:
                print("Alle forsøk feilet!")
                return None

# Test funksjonen
# robust_data = robust_weather_request(59.9139, 10.7522)
# print(robust_data)

### Bonusoppgave 3: Værvarsel med nedbør

Yr.no API-et inneholder også informasjon om nedbør. Utforsk datastrukturen og hent ut nedbørsprognoser.

In [ ]:
# Utforsk 'next_1_hours' og 'next_6_hours' data
# Disse inneholder nedbørsprognoser

def get_precipitation_forecast(lat, lon):
    """Hent nedbørsprognose for de neste timene"""
    
    # Din kode her:
    # Tips: Se på 'next_1_hours' og 'next_6_hours' i datastrukturen
    # Disse inneholder 'precipitation_amount' og 'precipitation_amount_max'
    
    pass

# Test funksjonen
# precipitation = get_precipitation_forecast(59.9139, 10.7522)
# print(precipitation)

## Refleksjonsspørsmål

Ta deg tid til å tenke over det du har lært:

1. **API-forståelse**: Hva er forskjellen mellom et API og en vanlig nettside?

2. **Feilhåndtering**: Hvorfor er det viktig å håndtere feil når man arbeider med API-er?

3. **Datastruktur**: Yr.no returnerer mye data. Hvordan bestemte du hvilke verdier som var viktigst å hente ut?

4. **Etikk**: Hvilke etiske hensyn bør man ta når man bruker andres API-er?

5. **Utvidelser**: Hvilke andre funksjoner kunne du tenke deg å legge til i en værapp?

Skriv ned tankene dine i cellen under:

**Mine refleksjoner:**

<!-- Skriv dine tanker her -->



## Oppsummering

🎉 **Gratulerer! Du har nå lært:**

✅ **REST API-konsepter**
- Hva et API er og hvordan det fungerer
- HTTP-metoder (GET, POST, etc.)
- Status koder (200, 404, etc.)

✅ **requests-biblioteket**
- Gjøre HTTP-forespørsler
- Håndtere JSON-data
- Sette headers og parametere

✅ **Yr.no API**
- Hente værdata for spesifikke koordinater
- Forstå komplekse datastrukturer
- Hente både nåværende vær og prognoser

✅ **Databehandling**
- Pakke ut relevant informasjon fra API-svar
- Sammenligne data fra flere kilder
- Lage sammendrag og statistikk

✅ **Feilhåndtering**
- Try/except blokker
- Håndtere nettverksfeil
- Validere data

### Neste steg i læringen:

🔗 **Utforsk andre API-er:**
- [JSONPlaceholder](https://jsonplaceholder.typicode.com/) - Test-API
- [OpenWeatherMap](https://openweathermap.org/api) - Alternativt vær-API
- [REST Countries](https://restcountries.com/) - Landeinformasjon

📚 **Lær mer om:**
- API-autentisering (API-nøkler, OAuth)
- GraphQL (alternativ til REST)
- Rate limiting og caching
- Asynkron programmering med `asyncio`

🛠️ **Prosjektideer:**
- Bygg en værapp med grafisk brukergrensesnitt
- Lag en bot som sender daglige værmeldinger
- Kombiner flere API-er (vær + kart + nyheter)
- Lag en værlogger som lagrer data i en database

### Viktige ressurser:

- 📖 [Requests dokumentasjon](https://docs.python-requests.org/)
- 🌤️ [Yr.no API dokumentasjon](https://api.met.no/weatherapi/)
- 🔧 [HTTP statuskoder](https://httpstatuses.com/)
- 📊 [Pandas dokumentasjon](https://pandas.pydata.org/docs/)

**Lykke til med videre utforskning av API-verdenen! 🚀**